# https://github.com/jbloomAus/DecisionTransformerInterpretability/issues/33


In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('arena-ldn/PPO-MiniGrid/Test-PPO-LSTM_checkpoints:v16', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: jbloomaus (arena-ldn). Use `wandb login --relogin` to force relogin


wandb:   11 of 11 files downloaded.  


In [7]:

import torch
import gymnasium as gym 
from src.environments.environments import make_env
from src.config import LSTMModelConfig, EnvironmentConfig, RunConfig, ConfigJsonEncoder
from src.ppo.agent import load_saved_checkpoint
from src.environments.registration import register_envs

register_envs()

path = "../artifacts/Test-PPO-LSTM_checkpoints:v16/Test-PPO-LSTM_10.pt"
agent = load_saved_checkpoint(path, 10)


/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment DynamicObstaclesMultiEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment CrossingMultiEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment MultiRoomMultiEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-package

In [4]:
import plotly.express as px 
obs, info = agent.envs.reset()
render = agent.envs.envs[0].env.render()
px.imshow(render)

# Goals

1. Assess the checkpoint performance
- Report on the episodic return and episode lengths
2. Sample trajectories for training of the decision transformer model
- Generate a trajectory file



In [6]:
import sys 
sys.path.append("..")
from src.ppo.agent import load_all_agents_from_checkpoints, sample_from_agents
from src.environments.registration import register_envs
register_envs()

checkpoint_folder_path = "../artifacts/Test-PPO-LSTM_checkpoints:v16"
agents = load_all_agents_from_checkpoints(checkpoint_folder_path)
all_episode_lengths, all_episode_returns = sample_from_agents(agents, rollout_length=300, num_envs=16)

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment DynamicObstaclesMultiEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment CrossingMultiEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment MultiRoomMultiEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-package

In [7]:

import pandas as pd
import plotly.graph_objects as go

def plot_checkpoints_stats(all_episode_lengths, all_episode_returns):

    # stack df's but label them with the agent number
    lengths = pd.concat(all_episode_lengths, keys=range(len(all_episode_lengths))).reset_index(level=0, drop=False)
    # rename level 0 to checkpoint
    lengths = lengths.rename(columns={'level_0': 'checkpoint'})

    # Compute mean and std of episode lengths
    length_stats = lengths.groupby('checkpoint')['episode_length'].agg(['mean', 'std']).reset_index()

    # histogram of all episode lengths
    fig1 = go.Figure()
    for idx, stats in length_stats.iterrows():
        fig1.add_trace(go.Box(y=lengths[lengths['checkpoint'] == stats['checkpoint']]['episode_length'],
                              name=f"{stats['checkpoint']}: ({stats['mean']:.2f}, +- {stats['std']:.2f})",
                              boxpoints='all', jitter=0.3, pointpos=-1.8))

    fig1.update_layout(title='Episode Lengths')

    # stack df's but label them with the agent number
    returns = pd.concat(all_episode_returns, keys=range(len(all_episode_lengths))).reset_index(level=0, drop=False)
    # rename level 0 to checkpoint
    returns = returns.rename(columns={'level_0': 'checkpoint'})

    # Compute mean and std of episode returns
    return_stats = returns.groupby('checkpoint')['episode_return'].agg(['mean', 'std']).reset_index()

    # histogram of all episode lengths
    fig2 = go.Figure()
    for idx, stats in return_stats.iterrows():
        fig2.add_trace(go.Box(y=returns[returns['checkpoint'] == stats['checkpoint']]['episode_return'],
                              name=f"{stats['checkpoint']}: ({stats['mean']:.2f}, +- {stats['std']:.2f})",
                              boxpoints='all', jitter=0.3, pointpos=-1.8))

    fig2.update_layout(title='Episode Returns')

    return fig1, fig2


trajectory_path = "rollouts"
num_envs = 5
# all_episode_lengths, all_episode_returns = sample_from_agents(agents, rollout_length=2000, trajectory_path=trajectory_path, num_envs=num_envs)
fig1, fig2 = plot_checkpoints_stats(all_episode_lengths, all_episode_returns)
fig1.show()
fig2.show()

## Let's see if we can train a model that on one of the best checkpoints

rollout agent 5 was pretty good so let's generate some data from that one and then call a Decision Transformer Training Script on it. 

In [9]:
import sys 
sys.path.append('..')
from src.ppo.agent import load_saved_checkpoint, process_memory_vars_to_log
from src.utils import TrajectoryWriter
from src.config import RunConfig, OnlineTrainConfig
from src.ppo.memory import Memory

trajectory_path = "../trajectories/MiniGrid-MemoryS7FixedStart-v0-Checkpoint11.gz"
agent = load_saved_checkpoint("../artifacts/Test-PPO-LSTM_checkpoints:v16/Test-PPO-LSTM_11.pt", 16)
num_envs = 16
rollout_length = 60000 # seems like a good amount
memory = Memory(agent.envs, OnlineTrainConfig(num_envs=num_envs), device=agent.device)
if trajectory_path:
    trajectory_writer = TrajectoryWriter(
        path= trajectory_path,
        run_config=RunConfig(track=False),
        environment_config=agent.environment_config,
        online_config=OnlineTrainConfig(num_envs=num_envs),
        model_config=agent.model_config
    )
else:
    trajectory_writer = None
agent.rollout(memory, rollout_length, agent.envs, trajectory_writer)
if trajectory_writer:
    trajectory_writer.tag_terminated_trajectories()
    trajectory_writer.write(upload_to_wandb=False)

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:87: UserWarning:

WARN: Overwriting existing videos at /Users/josephbloom/GithubRepositories/DecisionTransformerInterpretability/dev/videos/test folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)



Writing to ../trajectories/MiniGrid-MemoryS7FixedStart-v0-Checkpoint11.gz, using gzip compression
Trajectory written to ../trajectories/MiniGrid-MemoryS7FixedStart-v0-Checkpoint11.gz


In [ ]:
# let's load the trajectories and take a look at them:
# important questions:
# 1. are the trajectories terminated correctly?
# 2. are the trajectories the correct length?
# 3. of the ones that fail to succeed, is the RTG labelled  correctly? 



In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/DecisionTransformerInterpretability/MiniGrid-MemoryS7FixedStart-v0__Memory-VariedSamplingStrategies__1__1682558253:v0', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jbloom. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  
